In [ ]:
import os
import pandas as pd
from unidecode import unidecode
from dotenv import load_dotenv
import requests, zipfile
from io import StringIO, BytesIO

In [ ]:
load_dotenv()

# Extração e transformação inicial

Durante a evolução do projeto, também foi sugerida a inclusão do IDEB do município como indicador educacional ao dashboard de visão geral. Os dados são divulgados de maneira agregada para o município separados por anos iniciais, anos finais e ensino médio. Vamos obtê-los diretamente do site do INEP.

In [ ]:
def get_ideb(ciclo: str) -> pd.DataFrame:
    """
    Baixa e processa dados do IDEB para o município de São Paulo.
    
    Parâmetros:
    -----------
    ciclo : str
        O ciclo educacional desejado. Deve ser um dos valores:
        - 'anos_iniciais': Anos iniciais do ensino fundamental
        - 'anos_finais': Anos finais do ensino fundamental  
        - 'ensino_medio': Ensino médio
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame com os dados do IDEB filtrados para o município de São Paulo
        (código 3550308).
    
    Levanta:
    --------
    ValueError
        Se o parâmetro 'ciclo' não for um dos valores válidos.
    
    Exemplo:
    --------
    >>> df_ideb = get_ideb('anos_iniciais')
    >>> print(df_ideb.head())
    """
    if ciclo not in ['anos_iniciais', 'anos_finais', 'ensino_medio']:
        raise ValueError("Ciclo deve ser 'anos_iniciais', 'anos_finais' ou 'ensino_medio'")
    
    url = f'https://download.inep.gov.br/ideb/resultados/divulgacao_{ciclo}_municipios_2023.zip'
    with requests.get(url) as r:
        with zipfile.ZipFile(BytesIO(r.content)) as z:
            file_path = f'divulgacao_{ciclo}_municipios_2023/divulgacao_{ciclo}_municipios_2023.xlsx'
            df = pd.read_excel(z.open(file_path), skiprows=9)

    df = df[df['CO_MUNICIPIO'] == 3550308]
    return df

In [ ]:
df_ideb_iniciais = get_ideb('anos_iniciais')
df_ideb_iniciais

In [ ]:
df_ideb_finais = get_ideb('anos_finais')
df_ideb_finais

In [ ]:
df_ideb_medio = get_ideb('ensino_medio')
df_ideb_medio

# Transformação e mesclagem de dados

Primeiro, vamos concatenar os dados dos três níveis de ensino em um único DataFrame, adicionando uma coluna para identificar o nível de ensino correspondente.

In [ ]:
df_ideb = pd.concat(
    [df_ideb_iniciais
    .assign(CICLO='Anos Iniciais'),
    df_ideb_finais
    .assign(CICLO='Anos Finais'),
    df_ideb_medio
    .assign(CICLO='Ensino Médio')]
)

df_ideb

Agora, filtramos apenas a rede municipal.

In [ ]:
df_ideb = df_ideb[df_ideb['REDE'] == 'Municipal'].reset_index(drop=True)
df_ideb

Por último, mantemos apenas as colunas relevantes para o nosso dashboard e renomeamos as colunas para um formato mais amigável.

In [ ]:
df_ideb = df_ideb[['CICLO', 'VL_OBSERVADO_2023']]
df_ideb


In [ ]:
df_ideb = df_ideb.assign(IDEB_2023=pd.to_numeric(df_ideb['VL_OBSERVADO_2023']))
df_ideb = df_ideb.drop(columns='VL_OBSERVADO_2023')
df_ideb

# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [ ]:
base_path = os.path.join('data_output', 'educacao')

if not os.path.exists(base_path):
    os.makedirs(base_path)

filepath = os.path.join(base_path, f'ideb-municipal.csv')

df_ideb.to_csv(filepath,
               index=False,
               sep=';',
               decimal=',',
               encoding='utf-8')